<img src="logo.png">

# C06. Funciones y Procedimientos almacenados

Anteriormente ya hemos visto ejemplos de funciones en Postgresql. En este capítulo profundizaremos en su uso.

Un procedimiento almacenado es un programa, procedimiento o función, el cual está almacenado en la base de datos y listo para ser usado. Permite ejecutar instrucciones SQL almacenadas en una base de datos. 

Una de las características importantes es que no necesitan explícitamente devolver un valor, a diferencia de las funciones. Es importante comentar que Postgresql no tiene como tal procedimientos almacenados; sin embargo, nos permite crear funciones con el mismo comportamiento.


## Funciones

Recordemos la estructura básica de una función en Postgresql

``CREATE OR REPLACE FUNCTION Nombre(parámetros) RETURNS tipo 
AS 
$$
DECLARE
declaraciones de las variables;
BEGIN
código ejecutable
END
$$
LANGUAGE plpgsql``

Ya antes habíamos visto cómo hacer la función suma:

``CREATE OR REPLACE FUNCTION sumar(x int, y int) RETURNS int 
AS
$$
DECLARE z int;
BEGIN
z = x + y;
RETURN z;
END
$$
LANGUAGE plpgsql;``

Otra manera de mandar a llamar a los parámetros de la función es sin declararles un nombre:

``CREATE OR REPLACE FUNCTION multiplicar(int,int) RETURNS text 
AS
$$
DECLARE z int;
BEGIN
z = $1 * $2;
RETURN CONCAT('El producto de ',$1,' con ',$2,' es ',z);
END
$$
LANGUAGE plpgsql;``

## Procedimientos almacenados

Para crear un procedimiento almacenado, ocupamos la siguiente sintaxis:

``CREATE OR REPLACE PROCEDURE nombre_del_procedimiento(parámetros)
language plpgsql
as 
$$
begin
cuerpo;
end
$$;``

Para mandar a llamar el procedimiento, escribimos ``call nombre_del_procedimiento(argumentos)``

**Ejemplo.**

Consideremos la siguiente tabla:

``CREATE TABLE cuentas (
    id INT GENERATED BY DEFAULT AS IDENTITY,
    cliente VARCHAR(100) NOT NULL,
    balance numeric NOT NULL,
    PRIMARY KEY(id)
);``

``INSERT INTO cuentas (cliente,balance)
VALUES('Jorge',10000),
	  ('Héctor',10000),
	  ('Ramón',10000),
	  ('Luis',10000),
      ('José',10000),
      ('Sofía',10000),
      ('Leticia',10000),
      ('Gloria',10000);
``

El siguiente código crea un procedimiento almacenado llamado *tranferir* que transfiere una cantidad de dinero de una cuenta a otra. Mas precisamente: tomará la cantidad indicada por el tercer parámetro, lo quitará del cliente cuyo id es el primer parámetro y la pasará al balance del cliente cuyo id es es el segundo parámetro.

``create or replace procedure transferir(int, int, numeric)
language plpgsql    
as 
$$
begin
    update cuentas set balance = balance - $3 where id = $1;
    update cuentas set balance = balance + $3 where id = $2;
end;
$$;``

``call transferir(x,y,z)`` donde x,y,z son los parámetros

## Triggers

Un trigger  o disparador es un objeto que se almacena en nuestra base de datos y se asocia con una tabla en concreto. Éste trigger se ejecuta cuando sucede algún evento sobre la tabla que está asociada y no es necesario que un usuario lo ejecute

Para crear un trigger en PostgreSQL primero tenemos que crear una función donde el tipo que nos retorna dicha función sea un tipo trigger. A continuación mostramos la sintaxis que debemos de utilizar para crear este tipo de función que retorna un tipo trigger.

``CREATE FUNCTION nombre_funcion() RETURNS TRIGGER 
AS 
$$
BEGIN
acciones a ejecutar;
END;
$$ 
LANGUAGE plpgsql;``

Una vez que ya tenemos la función que va a ser ejecutada por el trigger, creamos este disparador donde le vamos a indicar cuándo debe activarse. En esta sintaxis se ejecutará una función antes de de cada inserción de datos en una tabla especificada.

``CREATE TRIGGER nombre_trigger BEFORE INSERT ON tabla FOR EACH ROW EXECUTE PROCEDURE nombre_funcion;``

**Ejemplo**

Supongamos que queremos mantener un respaldo de cómo era una tabla antes de actualizarla. Creamos primero una tabla con los campos que tiene *cuentas* y otros más:

``
create table cuentas_antes(
	id INT GENERATED BY DEFAULT AS IDENTITY,
    cliente VARCHAR(100) NOT NULL,
    balance numeric NOT NULL,
	usuario varchar(250) NOT NULL,
	Fecha date,
	tiempo time,
    PRIMARY KEY(id))
``

El siguiente disparador, llamado *antes_actualizar* se ejecutará antes de que se ejecute cualquier tipo de actualización en la tabla *clientes*, de tal manera que guardará en *cuentas_antes* los registros afectados directamente antes de hacer dicha actualización.    

``create or replace function revisar() returns trigger
as
$$
declare
    Usuario Varchar(250) := user; 
    Fecha date := current_date;
    Tiempo time := current_time;
begin
    INSERT INTO cuentas_antes values (old.id,old.cliente,old.balance, Usuario, Fecha, Tiempo);
    return new;
end
$$
language plpgsql``    

``create trigger antes_actualizar  before update on cuentas
for each row execute procedure revisar();``

**Ejemplo**

Supongamos ahora que queremos saber qué nuevos datos se han introducido. Creamos nuevamente una tabla con los campos que tiene *cuentas* y otros más:

``create table cuentas_ahora(
	id INT GENERATED BY DEFAULT AS IDENTITY,
    cliente VARCHAR(100) NOT NULL,
    balance numeric NOT NULL,
	usuario varchar(250) NOT NULL,
	Fecha date,
	tiempo time,
    PRIMARY KEY(id))``
	
El siguiente disparador, llamado *Actualizacion_reciente* se ejecutará después de que se ejecute cualquier insertado de datos en la tabla *clientes*, de tal manera que guardará en *cuentas_ahora* los nuevos registros. 

``create or replace function insertar() returns trigger
as
$$
declare
    Usuario Varchar(250) := user; 
    Fecha date := current_date;
    Tiempo time := current_time;
begin
    INSERT INTO cuentas_ahora values (new.id,new.cliente,new.balance, Usuario, Fecha, Tiempo);
    return new;
end
$$
language plpgsql``    

``create trigger Actualizacion_reciente  after insert on cuentas
for each row execute procedure insertar();``
